<a href="https://colab.research.google.com/github/mertege/FMCW-Data-Classification-/blob/main/FMCW_Data_CNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Spectrogram'lar üzerinden CNN koşturuluyor.
# 5-fold koşturuldu ve training datasına augmentation yapıldı. 
# Fast train data'sına slow'a göre 2 kat augmentation yapıldı.
# Validation ayrıldı ve early stopping özelliği kullanıldı.
# Mean test accuracy is 0.84, mean test f1 score is 0.82, max test accuracy is 0.90, max test f1 score is 0.89,
# min test accuracy is 0.79, min test f1 score is 0.75, std of test accuracy is 0.04, std of test f1 score is 0.05
# Time elapsed through all process: 290.14 sec

In [21]:
from keras.models import Sequential
from tensorflow.keras import layers
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense, BatchNormalization, Normalization
import tensorflow as tf
import scipy.io
import numpy as np
import cv2
import matplotlib.pyplot as plt
import random
from numpy.random import seed
from sklearn.model_selection import KFold, StratifiedKFold
import time
from sklearn.metrics import precision_recall_fscore_support
from keras.callbacks import EarlyStopping

In [2]:
pip install mat73

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
import mat73
spectrograms_shuffled = mat73.loadmat('/content/drive/MyDrive/spectrograms_shuffled.mat')
spectrograms_shuffled = spectrograms_shuffled['spectrograms_shuffled']
spectrograms_shuffled = np.transpose(spectrograms_shuffled, (2, 0, 1))
spectrograms_label_shuffled = scipy.io.loadmat('/content/drive/MyDrive/spectrograms_label_shuffled.mat')
spectrograms_label_shuffled = spectrograms_label_shuffled['spectrograms_label_shuffled']  

In [5]:
# Downscale Images
spectrograms_shuffled_downscaled = np.zeros((170,50,412))
for ii in range(spectrograms_shuffled.shape[0]):
  aa = np.float32(spectrograms_shuffled[ii,:,:])
  img = cv2.cvtColor(aa, cv2.COLOR_GRAY2BGR)
  img_gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
  spectrograms_shuffled_downscaled[ii,:,:] = cv2.resize(img_gray, dsize=(412, 50), interpolation=cv2.INTER_CUBIC)


In [6]:
del spectrograms_shuffled

In [7]:
# Add new axis
print(spectrograms_shuffled_downscaled.shape[1:])
spectrograms_shuffled_downscaled = spectrograms_shuffled_downscaled[:,:,:,np.newaxis] 
print(spectrograms_shuffled_downscaled.shape[1:])

(50, 412)
(50, 412, 1)


In [26]:

t = time.time()
# ---------------- Augmente and shuffle (train and test) data data ----------------
data_augmentation = tf.keras.Sequential([
  layers.RandomFlip("horizontal_and_vertical"),
  layers.RandomRotation(0.1),
])
# ---------------- Parameters ----------------
test_accuracy_per_fold = []
f1_score_per_fold = []
repeat_of_augmentation_for_fast = 9
repeat_of_augmentation_for_slow = np.floor(repeat_of_augmentation_for_fast/2)
repeat_of_augmentation_for_slow = int(repeat_of_augmentation_for_slow)
slow_size_of_validation = 20
fast_size_of_validation = slow_size_of_validation/2
fast_size_of_validation = int(fast_size_of_validation)
num_folds = 6
kfold = StratifiedKFold(n_splits=num_folds, shuffle=True, random_state = None) # random_state = 1 ile split run'dan run'a sabit.
epoch_number = 100
batch_size = 32

for train, test in kfold.split(spectrograms_shuffled_downscaled,spectrograms_label_shuffled):   
  randomlist_for_validation_indx = test
  randomlist_for_train_indx = train
  # test data
  spectrograms_shuffled_downscaled_test = spectrograms_shuffled_downscaled[randomlist_for_validation_indx,:,:,:]
  spectrograms_label_shuffled_test = spectrograms_label_shuffled[randomlist_for_validation_indx,:]
  #train data
  spectrograms_shuffled_downscaled_train = spectrograms_shuffled_downscaled[randomlist_for_train_indx,:,:,:]
  spectrograms_label_shuffled_train = spectrograms_label_shuffled[randomlist_for_train_indx,:]
  # ---------------- Split labels to equal them during augmentation ----------------
  slow_indexes = np.where(spectrograms_label_shuffled_train == 0)[0]
  fast_indexes = np.delete(range(0, spectrograms_label_shuffled_train.shape[0]), slow_indexes)
  slow_spectrograms_train_val = spectrograms_shuffled_downscaled_train[slow_indexes,:,:,:]
  fast_spectrograms_train_val = spectrograms_shuffled_downscaled_train[fast_indexes,:,:,:]
  # ---------------- Seperate Validation From Training Data ----------------
  # -- Slow --
  randomlist_for_validation_indx = random.sample(range(0, slow_spectrograms_train_val.shape[0]), slow_size_of_validation) # split validation data
  randomlist_for_train_indx = np.delete(range(0, slow_spectrograms_train_val.shape[0]), randomlist_for_validation_indx) # split training data
  slow_spectrograms_train = slow_spectrograms_train_val[randomlist_for_train_indx,:,:,:]
  slow_spectrograms_val = slow_spectrograms_train_val[randomlist_for_validation_indx,:,:,:]
  slow_label_val = np.zeros((slow_size_of_validation,1))
  # -- Fast -- 
  randomlist_for_validation_indx = random.sample(range(0, fast_spectrograms_train_val.shape[0]), fast_size_of_validation) # split validation data
  randomlist_for_train_indx = np.delete(range(0, fast_spectrograms_train_val.shape[0]), randomlist_for_validation_indx) # split training data
  fast_spectrograms_train = fast_spectrograms_train_val[randomlist_for_train_indx,:,:,:]
  fast_spectrograms_val = fast_spectrograms_train_val[randomlist_for_validation_indx,:,:,:]
  fast_label_val = np.ones((fast_size_of_validation,1))
  # -- Concat Fast and Slow Data at Validation Dataset -- 
  validation_spectrograms = np.concatenate((fast_spectrograms_val,slow_spectrograms_val),axis=0)
  validation_labels = np.concatenate((fast_label_val,slow_label_val),axis=0)
  # ---------------- Augmente Train Data for Fast ----------------
  size_of_samples_fast = fast_spectrograms_train.shape[0]
  augmented_image_fast = np.zeros((size_of_samples_fast*repeat_of_augmentation_for_fast,fast_spectrograms_train.shape[1],fast_spectrograms_train.shape[2],1))
  spectrograms_fast_label = np.ones((size_of_samples_fast*(repeat_of_augmentation_for_fast+1),1))
  for kk in range(repeat_of_augmentation_for_fast):
    for ii in range(size_of_samples_fast):
      augmented_image_fast[size_of_samples_fast*kk+ii,:,:,:] = data_augmentation(fast_spectrograms_train[ii,:,:,:])
  augmented_image_fast = np.concatenate((augmented_image_fast,fast_spectrograms_train),axis=0)
  # ---------------- Augmente Train Data for Slow ----------------
  size_of_samples_slow = slow_spectrograms_train.shape[0]
  augmented_image_slow = np.zeros((size_of_samples_slow*repeat_of_augmentation_for_slow,slow_spectrograms_train.shape[1],slow_spectrograms_train.shape[2],1))
  spectrograms_slow_label = np.zeros((size_of_samples_slow*(repeat_of_augmentation_for_slow+1),1))
  for kk in range(repeat_of_augmentation_for_slow):
    for ii in range(size_of_samples_slow):
      augmented_image_slow[size_of_samples_slow*kk+ii,:,:,:] = data_augmentation(slow_spectrograms_train[ii,:,:,:])
  augmented_image_slow = np.concatenate((augmented_image_slow,slow_spectrograms_train),axis=0)
  # ---------------- Concat Fast and Slow ----------------
  augmented_image = np.concatenate((augmented_image_fast,augmented_image_slow),axis=0)
  spectrograms_label_shuffled_concat = np.concatenate((spectrograms_fast_label,spectrograms_slow_label),axis=0)

  # ---------------- Normalize Inputs ----------------
  layer = Normalization(axis=None)
  layer.adapt(spectrograms_shuffled_downscaled_test)
  spectrograms_shuffled_downscaled_test = layer(spectrograms_shuffled_downscaled_test)
  layer = Normalization(axis=None)
  layer.adapt(validation_spectrograms)
  validation_spectrograms = layer(validation_spectrograms)
  layer = Normalization(axis=None)
  layer.adapt(augmented_image)
  augmented_image = layer(augmented_image)

  # ---------------- Neural Network Architecture ----------------
  model = Sequential()

  model.add(Conv2D(4, (3, 3), input_shape=augmented_image.shape[1:]))
  model.add(BatchNormalization())
  model.add(Activation('relu'))
  model.add(MaxPooling2D(pool_size=(2, 2)))

  model.add(Conv2D(8, (3, 3), input_shape=augmented_image.shape[1:]))
  model.add(BatchNormalization())
  model.add(Activation('relu'))
  model.add(MaxPooling2D(pool_size=(2, 2)))

  model.add(Conv2D(16, (3, 3)))
  model.add(BatchNormalization())
  model.add(Activation('relu'))
  model.add(MaxPooling2D(pool_size=(2, 2)))


  # model.add(Conv2D(64, (3, 3)))
  # model.add(BatchNormalization())
  # model.add(Activation('relu'))
  # model.add(MaxPooling2D(pool_size=(2, 2)))

  model.add(Flatten())  
  model.add(Dense(32))
  model.add(Activation('relu'))
  model.add(Dropout(0.5))
  model.add(Dense(1))
  model.add(Activation('sigmoid'))

  print(model.summary())
  # ---------------- Compile and Fit ----------------
  model.compile(loss='binary_crossentropy',
                optimizer='rmsprop',
                metrics=['accuracy'])
  
  earlyStopping = EarlyStopping(monitor='val_loss', patience=25, verbose=0,restore_best_weights=True, mode='min')
  # earlyStopping = EarlyStopping(monitor='val_accuracy', patience=15, verbose=0,restore_best_weights=True, mode='max')
  history = model.fit((augmented_image),(spectrograms_label_shuffled_concat),
                  epochs=epoch_number,
                  batch_size=batch_size,
                  shuffle = True,
                  callbacks=[earlyStopping],
                  validation_data = ((validation_spectrograms) , (validation_labels)))
  tf.keras.models.load_model
  test_loss, test_accuracy  = model.evaluate((spectrograms_shuffled_downscaled_test),(spectrograms_label_shuffled_test),
                batch_size=batch_size)
  # ---------------- Get Test Results ----------------
  y_test_predicted = model.predict((spectrograms_shuffled_downscaled_test), batch_size=batch_size)
  # ----- Binarize y_test_predicted values -----
  y_test_predicted_binary = np.zeros(y_test_predicted.size)
  for ii in range(y_test_predicted.size):
    if y_test_predicted[ii] < 0.5:
      y_test_predicted_binary[ii] = 0
    else:
      y_test_predicted_binary[ii] = 1
  
  test_precision, test_recall, test_f1_score, support = precision_recall_fscore_support(spectrograms_label_shuffled_test, y_test_predicted_binary, average='macro')

  test_accuracy_per_fold.append(test_accuracy)
  f1_score_per_fold.append(test_f1_score)

print(f'Mean test accuracy is {"{:.2f}".format(sum(test_accuracy_per_fold)/num_folds)}, mean test f1 score is {"{:.2f}".format(sum(f1_score_per_fold)/num_folds)}, \
max test accuracy is {"{:.2f}".format(max(test_accuracy_per_fold))}, max test f1 score is {"{:.2f}".format(max(f1_score_per_fold))}, \
min test accuracy is {"{:.2f}".format(min(test_accuracy_per_fold))}, min test f1 score is {"{:.2f}".format(min(f1_score_per_fold))}, \
std of test accuracy is {"{:.2f}".format(np.std(test_accuracy_per_fold, axis=0))}, std of test f1 score is {"{:.2f}".format(np.std(f1_score_per_fold, axis=0))}')
elapsed = time.time() - t
#  326.78 sec
print(f'Time elapsed through all process: {"{:.2f}".format(elapsed)}')

Model: "sequential_45"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_101 (Conv2D)         (None, 48, 410, 4)        40        
                                                                 
 batch_normalization_101 (Ba  (None, 48, 410, 4)       16        
 tchNormalization)                                               
                                                                 
 activation_169 (Activation)  (None, 48, 410, 4)       0         
                                                                 
 max_pooling2d_101 (MaxPooli  (None, 24, 205, 4)       0         
 ng2D)                                                           
                                                                 
 conv2d_102 (Conv2D)         (None, 22, 203, 8)        296       
                                                                 
 batch_normalization_102 (Ba  (None, 22, 203, 8)     

In [ ]:
# Test Amaçlı
model.predict(augmented_image)

In [ ]:
# Test Amaçlı
model.predict(spectrograms_shuffled_downscaled_test)